In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from skimage import measure

from loss_functions import *

In [ ]:

def custom_loss_proposed(weight_map):
    def dice_loss(y_true, y_pred, smooth = 1.):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        weights_f = K.flatten(weight_map)
        intersection = K.sum(weights_f * y_true_f * y_pred_f)
        score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
        return 1. - score
    return dice_loss 


In [ ]:
def custom_loss_proposed(weight_map):
    def tversky(y_true, y_pred, smooth = 1.):
        y_true_pos = K.flatten(y_true)
        y_pred_pos = K.flatten(y_pred)
        weights_f = K.flatten(weight_map)
        true_pos = K.sum(weights_f * y_true_pos * y_pred_pos)
        false_neg = K.sum(y_true_pos * (1-y_pred_pos))
        false_pos = K.sum((1-y_true_pos)*y_pred_pos)
        alpha = 0.1
        score =  (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
        return 1. - score
    return tversky 


In [ ]:
#ALL PARAMS

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7" #Use 1 gpu only

train = 1

useBound = False #use boundary GS as target

model_name = 'Unet_Tubule_Selahattin' 
imgpath = ('/media/hdd3/gunduz/lossFunction/images/Tubule/') 


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, concatenate

from keras.layers.core import Layer, Dense, Activation, Flatten, Reshape, Permute, Lambda
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
# from keras.layers.convolutional import ZeroPadding2D
# from keras.layers.convolutional import Convolution1D, MaxPooling1D
# from keras.layers.recurrent import LSTM
# from keras.layers.advanced_activations import LeakyReLU
from keras import optimizers
# from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import objectives
from keras import losses

def weighted_mae_loss(y_true, y_pred):
    print(y_true.shape)
    print(y_pred.shape)
    
    mae_loss = losses.mean_absolute_error(y_true, y_pred)
    print(mae_loss.shape)
    
    weight = K.cast(y_true >= 0.5, dtype='float32') * K.variable(np.array([1.0, 1.0, 1.0, 1.0, 1.0]))
    print(weight.shape)
    
    weight = K.sum(weight, axis=-1)
    print(weight.shape)
    
    out = mae_loss * weight
    print(out.shape)
    
    return out

def weighted_binary_crossentropy(p, w = 1):# Weighted categorical_crossentropy loss
    #p should contain pixel-wise weigth matrix
    #w is a floating point constant
    
    def loss(y_true, y_pred):        
        cc_loss = objectives.binary_crossentropy(y_true, y_pred)
        loss = (w * p * cc_loss)
        return K.mean(loss)
    return loss

def custom_loss(weight_map):
    def loss(y_true, y_pred):
        ce = losses.binary_crossentropy(y_true, y_pred)
        return np.dot(weight_map, ce)
    return loss 


#Segmentation Model
def unet_model(input_height=528, input_width=784, k=32, nChannels = 3, dropout_rate = 0.2):
    
    optimizer = optimizers.Adadelta()
    image = Input(shape= (input_height, input_width, nChannels))
    weights = Input(shape= (input_height, input_width))
    
    conv1 = Conv2D(k,(3, 3), activation='relu', padding='same')(image)
    conv1 = Dropout(dropout_rate)(conv1)
    conv1 = Conv2D(k, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(k*2,(3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(dropout_rate)(conv2)
    conv2 = Conv2D(k*2,(3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(k*4,(3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(dropout_rate)(conv3)
    conv3 = Conv2D(k*4,(3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(k*8,(3, 3), activation='relu', padding='same')(pool3)
    conv4 = Dropout(dropout_rate)(conv4)
    conv4 = Conv2D(k*8,(3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(k*16,(3, 3), activation='relu', padding='same')(pool4) 
    conv5 = Dropout(dropout_rate)(conv5)
    conv5 = Conv2D(k*16,(3, 3), activation='relu', padding='same')(conv5)
    
    conv5_ = Conv2D(k*32,(3, 3), activation='relu', padding='same')(conv5) 
    conv5_ = Dropout(dropout_rate)(conv5_) #new
    conv5_ = Conv2D(k*32,(3, 3), activation='relu', padding='same')(conv5_)
    
    conv5_2 = Conv2D(k*32,(3, 3), activation='relu', padding='same')(conv5_)
    conv5_2 = Dropout(dropout_rate)(conv5_2) #new 2
    conv5_2 = Conv2D(k*32,(3, 3), activation='relu', padding='same')(conv5_2)
    
    concat_1 = concatenate([conv5_2, conv5_], axis=3) #new 
    conv6_2 = Conv2D(k*16,(3, 3), activation='relu', padding='same')(concat_1)
    conv6_2 = Dropout(dropout_rate)(conv6_2) 
    conv6_2 = Conv2D(k*16, (3, 3), activation='relu', padding='same')(conv6_2)

    concat_2 = concatenate([conv6_2, conv5], axis=3)
    conv6_ = Conv2D(k*16,(3, 3), activation='relu', padding='same')(concat_2)
    conv6_ = Dropout(dropout_rate)(conv6_)
    conv6_ = Conv2D(k*16, (3, 3), activation='relu', padding='same')(conv6_)
 
    up1 = concatenate([UpSampling2D(size=(2, 2))(conv6_), conv4], axis=3)    
    conv6 = Conv2D(k*8,(3, 3), activation='relu', padding='same')(up1)
    conv6 = Dropout(dropout_rate)(conv6)
    conv6 = Conv2D(k*8, (3, 3), activation='relu', padding='same')(conv6)
    
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    conv7 = Conv2D(k*4,(3, 3), activation='relu', padding='same')(up2)
    conv7 = Dropout(dropout_rate)(conv7)
    conv7 = Conv2D(k*4, (3, 3), activation='relu', padding='same')(conv7)
    
    up3 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    conv8 = Conv2D(k*2,(3, 3), activation='relu', padding='same')(up3)
    conv8 = Dropout(dropout_rate)(conv8)
    conv8 = Conv2D(k*2, (3, 3), activation='relu', padding='same')(conv8)
    
    up4 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    conv9 = Conv2D(k,(3, 3), activation='relu', padding='same')(up4)
    conv9 = Dropout(dropout_rate)(conv9)
    conv9 = Conv2D(k, (3, 3), activation='relu', padding='same')(conv9)

    output = Conv2D(1, (1,1), activation = 'sigmoid')(conv9)
   

    model = Model(inputs = [image, weights], outputs = output )
    
#     loss1 = weighted_binary_crossentropy(weights, w = 1)
#     loss2 = weighted_categorical_crossentropy(out2_weights, w = 1)
    loss1 = custom_loss(weights)
#     loss2 = custom_loss_proposed(weights)
   
    
    model.compile(loss = tversky_loss, optimizer = optimizer , metrics=['binary_accuracy'])
    
#     model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['binary_accuracy'])
    
    return model

In [ ]:
# read images
from glob import glob
import scipy
from scipy import io

gs_images = []
images = []
wei_images = []

im_width = 784
im_height = 528

path_images = sorted(glob('/media/hdd3/gunduz/cansari/datasets/tubule/images/*'))

for img_path in path_images:
    
    print(img_path)
    img = scipy.misc.imread(img_path)
    gs = scipy.io.loadmat(img_path.replace("images","gold_standard_segm").replace(".jpg",".mat"))
    gs = (gs["goldSeg"] / 255).astype(np.int_)
    images.append(img)
    gs_images.append(gs)
    
    wei = np.zeros(shape = (im_height,im_width), dtype=float)
    weight = sum(sum(gs[:,:,0])) / (im_height*im_width)
    
    if weight < 0.5:
        wei[gs[:,:,0] > 0] = (1.0 - weight) + 0.0001
        wei[gs[:,:,0] == 0] = (weight) + 0.0001
        wei_images.append(wei)
        
    else:
        wei[gs[:,:,0] > 0] = (weight) + 0.0001
        wei[gs[:,:,0] == 0] = (1.0 - weight) + 0.0001
        wei_images.append(wei)

In [ ]:
for i in range(85):
    fig=plt.figure(figsize=(10, 3))

    fig.add_subplot(1, 3, 1)
    plt.imshow(images[i])  
    
    gs = gs_images[i]
    fig.add_subplot(1, 3, 2)
    plt.imshow(gs[:,:,0], cmap="gray")
    
    wei = wei_images[i]
    fig.add_subplot(1, 3, 3)
    plt.imshow(wei, cmap="gray")    

In [ ]:
plt.imshow(images[2])

In [ ]:
gs = gs_images[2]
plt.imshow(gs[:,:,0], cmap="gray")

In [ ]:
print(gs.shape)
print(gs[:,:,0])
print(sum(sum(gs[:,:,0])))
weight = sum(sum(gs[:,:,0])) / (im_height*im_width)
print(weight)

In [ ]:
wei = wei_images[2] 
plt.imshow(wei, cmap="gray")

In [ ]:
print(wei[400,100])
print(wei[200,100])
print(wei[400,100] + wei[200,100])

In [ ]:
# separating train, validation and test sets

import random 
random.seed(1)
randomlist = random.sample(range(0, 85), 85)

trSize = 60
valSize = 15
tsSize = 10

images_train =  []
images_val =  []
images_test =  []

gs_train =  []
gs_val =  []
gs_test =  []

wei_train = []
wei_val = []
wei_test = []

for index in randomlist:
    trC = len(images_train)
    valC = len(images_val)
    tsC = len(images_test)

    if trC < trSize:
        images_train.append(images[index])
        gs_train.append(gs_images[index])
        wei_train.append(wei_images[index])
        
    if trC == trSize and valC < valSize:
        images_val.append(images[index])
        gs_val.append(gs_images[index])
        wei_val.append(wei_images[index])
        
    if trC == trSize and valC == valSize:
        images_test.append(images[index])
        gs_test.append(gs_images[index])
        wei_test.append(wei_images[index])

In [ ]:
plt.imshow(images_val[0])

In [ ]:
gs = gs_val[0]
plt.imshow(gs[:,:,0], cmap="gray")

In [ ]:
wei = wei_val[0] 
plt.imshow(wei, cmap="gray")

In [ ]:
# Final Datasets

images_train =   np.asarray(images_train)
gs_train = np.asarray(gs_train)
wei_train = np.asarray(wei_train)

images_val =   np.asarray(images_val)
gs_val = np.asarray(gs_val)
wei_val = np.asarray(wei_val)
    
images_test =   np.asarray(images_test)
gs_test = np.asarray(gs_test)
wei_test = np.asarray(wei_test)

#normalize

def normalizeImg(img):
    norm_img = np.zeros(img.shape)
    for i in range(3):
        norm_img[:,:,i] = (img[:,:,i] - img[:,:,i].mean()) / (img[:,:,i].std())
    return norm_img

x_train = np.zeros(images_train.shape)
x_valid = np.zeros(images_val.shape)
x_test = np.zeros(images_test.shape)
    
for i in range(images_train.shape[0]):
    x_train[i,:,:,:] = normalizeImg(images_train[i,:,:,:])
    
for i in range(images_val.shape[0]):
    x_valid[i,:,:,:] = normalizeImg(images_val[i,:,:,:])
    
for i in range(images_test.shape[0]):
    x_test[i,:,:,:] = normalizeImg(images_test[i,:,:,:])

y_train = (gs_train > 0).astype(np.int_)
y_valid = (gs_val > 0).astype(np.int_)
y_test = (gs_test > 0).astype(np.int_)

In [ ]:
y_train=y_train[:,:,:,0].reshape(trSize,im_height ,im_width, 1)
y_valid=y_valid[:,:,:,0].reshape(valSize,im_height ,im_width, 1) 
y_test=y_test[:,:,:,0].reshape(tsSize,im_height ,im_width, 1)

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)
print(wei_train.shape)
print(wei_val.shape)
print(wei_test.shape)

In [ ]:
wei_train_2 = np.full((wei_train.shape), 0.1)
wei_val_2 = np.full((wei_val.shape), 0.1)


for i in range (len(y_train)):
    out_masks = y_train[i,:,:,0] == 1
    wei_train_2[i][out_masks] = 0.9

for i in range (len(y_valid)):
    out_masks = y_valid[i,:,:,0] == 1
    wei_val_2[i][out_masks] = 0.9
    

In [ ]:
for i in range(20):
    fig=plt.figure(figsize=(10, 10))

    fig.add_subplot(1, 2, 1)
    plt.imshow(images_train[i,:,:,:])

    fig.add_subplot(1, 2, 2)
    plt.imshow(y_train[i,:,:,0], cmap="gray")

In [ ]:
count = 0
index = []
for i in range(60):
    if y_train[i,:,:,0].max() == 0:
        count += 1
        index.append(i)
print(count)
print(index)

In [ ]:
model_path = ('/media/hdd3/gunduz/lossFunction/models/%s.hdf5' % model_name) 
print(model_path)

model =  unet_model(input_height=528, input_width=784, k=32, nChannels = 3, dropout_rate = 0.2)
model.summary()

In [ ]:
from tensorflow import keras
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
if(train):

    #Train the model
    checkpointer = ModelCheckpoint(model_path, monitor='val_loss', verbose=0,
                                   save_best_only=True, save_weights_only=False, mode='auto', period=1)
    earlystopper = EarlyStopping(patience = 100, verbose=1)
    
    hist = model.fit(x = [x_train, wei_train], y = y_train, 
                     validation_data = ([x_valid, wei_val], y_valid),
                     epochs = 1000, batch_size = 2, shuffle=True, # sample_weight = {'output_': wei_train[:,:,0]},                
                     callbacks=[checkpointer, earlystopper])

    # summarize history for loss
    fig = plt.figure()
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

else:
    model.load_weights(model_path)

In [ ]:
# test the model

wei_test_1 = np.ones((x_test.shape[0], x_test.shape[1], x_test.shape[2]))
wei_test_1 = wei_test_1 / 2
pr_test = model.predict([x_test, wei_test_1], batch_size =4, verbose = 1)

print('pr_test.shape : ', pr_test.shape)

In [ ]:
# loss_fuction = "wbce_"
# loss_fuction = "focal_b2_"
loss_fuction = "tversky_2_"
# loss_fuction = "focal_tversky_"
# loss_fuction = "compound_"
# loss_fuction = "bce_"

for i in range(10):
    scipy.io.savemat(imgpath + loss_fuction + str(i) + ".mat", {'im': images_test[i,:,:,:], 'gs': gs_test[i,:,:,0], 'pr': pr_test[i,:,:,0]})
    print(i)

In [ ]:
for i in range(10):
    fig=plt.figure(figsize=(10, 10))

    fig.add_subplot(1, 3, 1)
    plt.imshow(images_test[i,:,:,:] / 255.0)

    fig.add_subplot(1, 3, 2)
    #plt.imshow(gs_test[i,:,0].reshape(1024, 1024), cmap="gray")
    plt.imshow(y_test[i,:,:,0], cmap="gray")

    fig.add_subplot(1, 3, 3)
    #plt.imshow(pr_test[i,:,0].reshape(1024, 1024), cmap="gray")
    plt.imshow(pr_test[i,:,:,0], cmap="gray")

In [ ]:
out_masks = pr_test[:,:,:,0] > 0.5
new_data = np.zeros(out_masks.shape)
new_data[out_masks] = 1

sc = []
for i in range (len(new_data)):
    TP, FP, TN, FN = perf_measure(gs_test[i,:,:,0].astype(int), new_data[i,:,:].astype(int))
    sc.append([dice(TP, FP, FN), sensitivity(TP, FN), specificity(FP, TN), precision(TP, FP)])
sc = np.asarray(sc)

sc.mean(axis=0).round(4)

In [ ]:
# loading predicted images

img = []
gold_std = []
predict_segm = []

path_images = sorted(glob(imgpath + "*"))

for img_path in path_images:    
    
    image = scipy.io.loadmat(img_path)
    img.append(image["im"])
    gold_std.append(image["gs"])
    predict_segm.append(image["pr"])
    
img = np.asarray(img)
gold_std = np.asarray(gold_std)
predict_segm = np.asarray(predict_segm)
    
# for i in range(len(img)):
#     fig=plt.figure(figsize=(10, 10))

#     fig.add_subplot(1, 3, 1)
#     plt.imshow(img[i,:,:,:] / 255.0)

#     fig.add_subplot(1, 3, 2)
#     #plt.imshow(gs_test[i,:,0].reshape(1024, 1024), cmap="gray")
#     plt.imshow(gold_std[i,:,:], cmap="gray")

#     fig.add_subplot(1, 3, 3)
#     #plt.imshow(pr_test[i,:,0].reshape(1024, 1024), cmap="gray")
#     plt.imshow(pr_weighted_ce[i,:,:], cmap="gray")

In [ ]:
out_masks = predict_segm[:,:,:] > 0.5
new_data = np.zeros(out_masks.shape)
new_data[out_masks] = 1

In [ ]:
for i in range(30,40):
    fig=plt.figure(figsize=(10, 10))

    fig.add_subplot(1, 3, 1)
    plt.imshow(img[i,:,:,:] / 255.0)

    fig.add_subplot(1, 3, 2)
    plt.imshow(gold_std[i,:,:], cmap="gray")

    fig.add_subplot(1, 3, 3)
    plt.imshow(new_data[i,:,:], cmap="gray")

In [ ]:
import cv2 as cv

def connected_comp(img):
    
    num_labels, labels = cv.connectedComponents(img.astype("uint8"))

    # Map component labels to hue val, 0-179 is the hue range in OpenCV
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv.merge([label_hue, blank_ch, blank_ch])

    # Converting cvt to BGR
    labeled_img = cv.cvtColor(labeled_img, cv.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0
    
    return labeled_img    

In [ ]:
for i in range(10,20):
    fig=plt.figure(figsize=(20, 10))

    fig.add_subplot(1, 6, 1)
    plt.imshow(img[i,:,:,:] / 255.0), plt.title("Image"), plt.xticks([]), plt.yticks([])

    fig.add_subplot(1, 6, 2)
    plt.imshow(gold_std[i,:,:], cmap="gray"), plt.title("Ground Truth"), plt.xticks([]), plt.yticks([])

    fig.add_subplot(1, 6, 3)
    labeled_img = connected_comp(new_data[i,:,:])
    plt.imshow(cv.cvtColor(labeled_img, cv.COLOR_BGR2RGB)), plt.xticks([]), plt.yticks([]), plt.title("Focal Loss")
    
    fig.add_subplot(1, 6, 4)
    labeled_img = connected_comp(new_data[i+10,:,:])
    plt.imshow(cv.cvtColor(labeled_img, cv.COLOR_BGR2RGB)), plt.xticks([]), plt.yticks([]), plt.title("Focal Tversky")
    
    fig.add_subplot(1, 6, 5)
    labeled_img = connected_comp(new_data[i+20,:,:])
    plt.imshow(cv.cvtColor(labeled_img, cv.COLOR_BGR2RGB)), plt.xticks([]), plt.yticks([]), plt.title("Tversky (β=0.88)")
    
    fig.add_subplot(1, 6, 6)
    labeled_img = connected_comp(new_data[i+30,:,:])
    plt.imshow(cv.cvtColor(labeled_img, cv.COLOR_BGR2RGB)), plt.xticks([]), plt.yticks([]), plt.title("Weighted Binary CE")
    
# plt.savefig('prediction.png')

In [ ]:
def perf_measure(y_target, y_pred):
    
    TP = np.sum((y_target==1) & (y_pred==1))
    FP = np.sum((y_pred==1) & (y_target!=y_pred))    
    TN = np.sum((y_target==0) & (y_pred==0))
    FN = np.sum((y_pred==0) & (y_target!=y_pred))
    
    return TP, FP, TN, FN

def sensitivity(TP, FN):
    return (TP / (TP+FN)) 

def specificity(FP, TN):
    return (TN / (TN+FP))

def accuracy(TP, FP, TN, FN):
    return ((TP+TN) / (TP + FP + TN + FN))

def precision(TP, FP):
    return (TP / (TP+FP))

def dice(TP, FP, FN):
    return ((2*TP) / (2*TP+FP+FN))

In [ ]:
metrics = []
for i in range (len(new_data)):
    TP, FP, TN, FN = perf_measure(gold_std[i,:,:].astype(int), new_data[i,:,:].astype(int))
    metrics.append([dice(TP, FP, FN), sensitivity(TP, FN), specificity(FP, TN), precision(TP, FP)])
metrics = np.asarray(metrics)

metrics.mean(axis=0).round(4)

In [ ]:
for i in range(int(len(metrics)/10)):
    index = i * 10 
    print(metrics[index:index+10].mean(axis=0).round(4))

In [ ]:
metrics[10:20]